### Inserting Temporal Data

mobilitydb-sqlalchemy lets you use pandas DataFrame (which are great for timeseries data) while you are in the Python world, and translates it back and for to temporal types defined in mobilitydb.

A point to note here is that we assume that the DataFrame’s columns are named “value” (except in case of TGeomPoint where it is “geometry”) and “t” for the data and the timestamp respectively.

Here we show how we can store numeric data which changes over time (i.e. tfloat), using the `mobilitydb_sqlalchemy.types.TFloat.TFloat` class.

Running the following code will create a new table with a tfloat column, and insert one row of hardcoded data into it.

In [19]:
import datetime
import pandas as pd

from mobilitydb_sqlalchemy import TFloat
from sqlalchemy import Column, Integer, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

**Setup the engine and session**, make sure you set the right url to connect to your mobilitydb instance.

In [20]:
engine = create_engine("postgresql://docker:docker@db:5432/mobilitydb", echo=True)
session = sessionmaker(bind=engine)()

**Setup and create the tables** (only one in our case here)

In [21]:
Base = declarative_base()

class TemporalFloats(Base):
    __tablename__ = "tfloat_test_002"
    tid = Column(Integer, primary_key=True)
    tdata = Column(TFloat(True, False))

Base.metadata.create_all(engine)

2021-04-23 16:48:26,608 INFO sqlalchemy.engine.Engine select version()
2021-04-23 16:48:26,609 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-23 16:48:26,618 INFO sqlalchemy.engine.Engine select current_schema()
2021-04-23 16:48:26,619 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-23 16:48:26,624 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-04-23 16:48:26,626 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-23 16:48:26,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-23 16:48:26,637 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-04-23 16:48:26,638 INFO sqlalchemy.engine.Engine [generated in 0.00156s] {'name': 'tfloat_test_002'}
2021-04-23 16:48:26,645 INFO sqlalchemy.engine.Engine COMMIT


**Prepare the data**

In [22]:
df = pd.DataFrame(
    [
        {"value": 0, "t": datetime.datetime(2018, 1, 1, 12, 0, 0)},
        {"value": 8.2, "t": datetime.datetime(2018, 1, 1, 12, 6, 0)},
        {"value": 6.6, "t": datetime.datetime(2018, 1, 1, 12, 10, 0)},
        {"value": 9.1, "t": datetime.datetime(2018, 1, 1, 12, 15, 0)},
    ]
).set_index("t")
df.head()

,value
t,
2018-01-01 12:00:00,0.0
2018-01-01 12:06:00,8.2
2018-01-01 12:10:00,6.6
2018-01-01 12:15:00,9.1


**Insert the Data**

In [23]:
row = TemporalFloats(tdata=df,)
session.add(row)
session.commit()

2021-04-23 16:48:30,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-23 16:48:30,367 INFO sqlalchemy.engine.Engine INSERT INTO tfloat_test_002 (tdata) VALUES (%(tdata)s) RETURNING tfloat_test_002.tid
2021-04-23 16:48:30,368 INFO sqlalchemy.engine.Engine [generated in 0.00435s] {'tdata': '[0@2018-01-01T12:00:00+0000, 8.2@2018-01-01T12:06:00+0000, 6.6@2018-01-01T12:10:00+0000, 9.1@2018-01-01T12:15:00+0000)'}
2021-04-23 16:48:30,380 INFO sqlalchemy.engine.Engine COMMIT
